In [11]:
# windows打印本地摄像头列表
import cv2
import os
import numpy as np

# 打印摄像头列表
cam_list = []
for i in range(10):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        cam_list.append(i)
        print("摄像头{}:".format(i))
        print("  可用")





摄像头0:
  可用
摄像头1:
  可用
摄像头2:
  可用


In [12]:
# 使用本地摄像头
cap = cv2.VideoCapture(1)
while True:
    ret, frame = cap.read()
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()